In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt


In [2]:
# Assigning the GPU for computation

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
#Performing some Transformation to the images

transform = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

In [4]:
#initializing the batch_size and loading the data

batch_size = 32
trainset = torchvision.datasets.CIFAR10(root='/content/CNN', train=True, transform=transform, download = True)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='/content/CNN', train = False, transform=transform, download=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)


100%|██████████| 170498071/170498071 [00:02<00:00, 83909038.86it/s] 


Extracting /content/CNN/cifar-10-python.tar.gz to /content/CNN
Files already downloaded and verified


In [5]:
# Hard coding the labels

classes = ['plane', 'car', 'bird','cat','deer','dog','frog', 'horse', 'ship','truck']

In [6]:
# Constructing the CNN Architecture

class CIFAR10(nn.Module):

  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(3,12,5)
    self.act1 = nn.ReLU()
    self.pool1 = nn.MaxPool2d(2)

    self.conv2 = nn.Conv2d(12,32,5)
    self.act2 = nn.ReLU()
    self.pool2 = nn.MaxPool2d(2)

    self.fc1 = nn.Linear(32*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, 10)

  def forward(self, x):
    x = self.pool1(self.act1(self.conv1(x)))
    x = self.pool2(self.act2(self.conv2(x)))
    x = torch.flatten(x,1)
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.fc3(x)
    return x


In [7]:
#Instantiating the Model, Defining the Loss and optimizer

model = CIFAR10()
model.to(device)  # Transferring the model to GPU
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [8]:
# Training the Model

running_loss = 0
n_epoch = 10
total_num_steps = len(trainloader)
for epoch in range(n_epoch):
  for i, (inputs, labels) in enumerate(trainloader):
    # Inputs are transferred to GPU
    inputs = inputs.to(device)
    labels = labels.to(device)

    y_pred = model(inputs)
    loss = loss_func(y_pred, labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if i % 1000 == 999:    # print every 1000 mini-batches
        print(f'[{epoch + 1}, {i + 1:5d}] loss: {loss}')
        loss = 0.0

print("Hooray, Finished Training")

[1,  1000] loss: 2.279642105102539
[2,  1000] loss: 1.8791576623916626
[3,  1000] loss: 1.7026714086532593
[4,  1000] loss: 1.6104748249053955
[5,  1000] loss: 1.257896065711975
[6,  1000] loss: 1.4259532690048218
[7,  1000] loss: 1.0964491367340088
[8,  1000] loss: 1.4295923709869385
[9,  1000] loss: 1.7202701568603516
[10,  1000] loss: 1.560673713684082
Hooray, Finished Training


In [9]:
# Testing the Model

correct = 0
total = 0
with torch.no_grad():
  for data in testloader:
    images, labels = data
    # Inputs are transferred to GPU
    images = images.to(device)
    labels = labels.to(device)
    output = model(images)
    _, predicted = torch.max(output, 1)
    total += labels.size(0)
    correct += (predicted ==  labels).sum().item()

print(f"The accuracy of the Model is {100* correct//total}%")




The accuracy of the Model is 57%


In [12]:
# Train accuracy the Model

correct = 0
total = 0
with torch.no_grad():
  for data in trainloader:
    images, labels = data
    # Inputs are transferred to GPU
    images = images.to(device)
    labels = labels.to(device)
    output = model(images)
    _, predicted = torch.max(output, 1)
    total += labels.size(0)
    correct += (predicted ==  labels).sum().item()

print(f"Training accuracy of the Model is {100* correct//total}%")




Training accuracy of the Model is 59%


In [10]:
# Viewing the image and the corresponding label for better understanding
'''
s = next(iter(trainloader))
inputs, labels = s
input_to_view = inputs[0]
label_to_view = labels[0]

image_to_view = input_to_view.numpy().transpose(1,2,0)
plt.imshow(image_to_view)
name = classes[label_to_view]
plt.title(f"Label : {name}")
plt.show()
'''

'\ns = next(iter(trainloader))\ninputs, labels = s\ninput_to_view = inputs[0]\nlabel_to_view = labels[0]\n\nimage_to_view = input_to_view.numpy().transpose(1,2,0)\nplt.imshow(image_to_view)\nname = classes[label_to_view]\nplt.title(f"Label : {name}")\nplt.show()\n'

In [11]:
model_path = '/content/CNN/cnn_part1.pth'
torch.save(model.state_dict, model_path)